In [1]:
import json
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

from amelia_datatools.utils.common import DATA_DIR, VERSION, OUTPUT_DIR

out_dir = f"{OUTPUT_DIR}/type"
os.makedirs(out_dir, exist_ok=True)


In [2]:
airport = "katl"
base_dir = DATA_DIR
traj_version = VERSION


traj_dir = os.path.join(base_dir, f'traj_data_{traj_version}', 'raw_trajectories', f'{airport}')


traj_files = [os.path.join(traj_dir, f) for f in os.listdir(traj_dir)]
print(traj_files[0])

data = pd.read_csv(traj_files[0])
data.head()

/Users/alonso.cano/Developer/CMU/Amelia/AmeliaDataTools/datasets/amelia/traj_data_a10v08/raw_trajectories/katl/KATL_498_1682107200.csv


,Frame,ID,Altitude,Speed,Heading,Lat,Lon,Range,Bearing,Type,Interp,x,y
0,0,17335176,1006.25,16.666667,270.582275,33.64849,-84.437928,1.606820,-0.619965,2.0,[ORG],1.307789,-0.933573
1,0,17336725,1006.25,0.000000,0.000000,33.64047,-84.420500,0.801022,1.021473,2.0,[ORG],0.418222,0.683174
2,0,17336862,1006.25,32.000000,69.713745,33.63601,-84.439076,1.043019,-1.644141,2.0,[ORG],-0.076431,-1.040215
3,0,17336755,1016.25,112.500000,90.800903,33.64954,-84.434894,1.566424,-0.429385,0.0,[ORG],1.424227,-0.652120
4,0,17336885,1006.25,0.000000,9.997559,33.64857,-84.438944,1.670340,-0.662811,2.0,[ORG],1.316672,-1.027820


In [3]:
agent_types = {}
total_count = 0
for f in tqdm(traj_files):
    data = pd.read_csv(f)

    unique_IDs = np.unique(data.ID)
    for ID in unique_IDs:
        atype = data[data.ID == ID].Type.astype(int)
        atype = atype[np.diff(atype, prepend=np.nan).astype(bool)].astype(str).tolist()
        key = ''.join(atype)

        if agent_types.get(key) is None:
            agent_types[key] = 0
            key_p = f"{key}_perc"
            agent_types[key_p] = 0.0

        agent_types[key] += 1
        total_count += 1


100%|██████████| 686/686 [09:56<00:00,  1.15it/s]


In [4]:

for k, v in agent_types.items():
    if "perc" in k: continue
    kp = f"{k}_perc"
    agent_types[kp] = round(v / total_count, 3)

agent_types['total'] = total_count

In [5]:
with open(f'{out_dir}/{airport}.json', 'w') as f:
    json.dump(agent_types, f, indent=2)